In [1]:
import numpy as np, requests, zipfile, StringIO, pandas as pd, json, copy

#suppres warnings
import warnings
warnings.simplefilter(action = "ignore")

In [2]:
import matplotlib.pyplot as plt

In [6]:
#z = zipfile.ZipFile('Forestry_Trade_Flows_E_All_Data.zip')   #stream web content of zip file to read
z = zipfile.ZipFile('../db/Forestry_Trade_Flows_E_All_Data2.zip')   #stream web content of zip file to read
df=pd.read_csv(z.open('Forestry_Trade_Flows_E_All_Data.csv'))

In [6]:
set(df[df['Reporter Countries']=='Romania']['Element'])

{'Export Quantity', 'Export Value', 'Import Quantity', 'Import Value'}

In [7]:
data={}
cs=set()
for i in df.T.iteritems():
    country=i[1][0]
    partner=i[1][2]
    cs.add(country)
    cs.add(partner)
    item=i[1][5]
    element=i[1][7]
    for k in range(9,44,2):
        if country not in data: data[country]=[]
        if not np.isnan(i[1][k]):
            if 'Value' in element: 
                year=int(df.columns[k][1:])
                if 'Import' in element: g="f"
                else: g="m"
                data[country].append({"partner":partner,
                                  "item":item,
                                  "g":g,
                                  "year":year,
                                  "s":i[1][k]})     

In [8]:
data={}
cs=set()
for i in df.T.iteritems():
    country=i[1][0]
    partner=i[1][2]
    cs.add(country)
    cs.add(partner)
    item=i[1][5]
    element=i[1][7]
    for k in range(9,44,2):
        if country not in data: data[country]={}
        if not np.isnan(i[1][k]):
            year=int(df.columns[k][1:])
            if 'Import' in element: g="f"
            else: g="m"
            if g not in data[country]:data[country][g]={}
            if partner not in data[country][g]:data[country][g][partner]={}
            if year not in data[country][g][partner]:data[country][g][partner][year]={}
            if item not in data[country][g][partner][year]:data[country][g][partner][year][item]={}
            if 'Value' in element: 
                data[country][g][partner][year][item]["s"]=i[1][k]
            else: 
                data[country][g][partner][year][item]["p"]=i[1][k]

In [9]:
data2={}
for country in data:
    if country not in data2:data2[country]=[]
    for g in data[country]:
        for partner in data[country][g]:
            for year in data[country][g][partner]:
                for item in data[country][g][partner][year]:
                    if (("s" in data[country][g][partner][year][item]) and ("p" in data[country][g][partner][year][item])):
                        data2[country].append({"partner":partner,
                                  "item":item,
                                  "g":g,
                                  "year":year,
                                  "s":data[country][g][partner][year][item]["s"],
                                  "p":data[country][g][partner][year][item]["p"]
                             })    

In [11]:
try:
    import zlib
    compression = zipfile.ZIP_DEFLATED
except:
    compression = zipfile.ZIP_STORED

In [12]:
for c in data2:
    file('dummy.json','w').write(json.dumps(data2[c])) 
    zf = zipfile.ZipFile('zips2/'+str(c)+'.zip', mode='w')
    zf.write('dummy.json','data.json',compress_type=compression)
    zf.close()

In [13]:
file('json/countries2.json','w').write(json.dumps(data.keys())) 

In [14]:
file('json/data2.json','w').write(json.dumps(data)) 

In [15]:
cdict={}
for i in df.T.iteritems():
    if i[1][0] not in cdict:cdict[i[1][0]]=i[1][1]
    if i[1][2] not in cdict:cdict[i[1][2]]=i[1][3]
    if i[1][1] not in cdict:cdict[i[1][1]]=i[1][0]
    if i[1][3] not in cdict:cdict[i[1][3]]=i[1][2]

In [16]:
cdict[107]=u"C\xf4te d'Ivoire"
cdict[u"C\xf4te d'Ivoire"]=cdict["C\xf4te d'Ivoire"]
cdict.pop("C\xf4te d'Ivoire")

107L

In [17]:
cdict[182]=u'R\xe9union'
cdict[u'R\xe9union']=cdict['R\xe9union']
cdict.pop('R\xe9union')

182L

In [18]:
file('json/cdict2.json','w').write(json.dumps(cdict)) 

In [19]:
countries=[]
for k in cdict:
    try: int(k)
    except: countries.append(k)
file('json/countries2.json','w').write(json.dumps(countries)) 

In [20]:
cdict[252]

'Unspecified Area'